In [52]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle

## Loading of the Datasets

In [5]:
filename = "diabetes.csv"
abs_filepath = os.path.join(os.getcwd(), filename)
df = pd.read_csv(abs_filepath)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.shape

(768, 9)

In [12]:
## Checking Null Values
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [15]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [16]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [18]:
df.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


## Train Test Split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Outcome"], axis=1), df["Outcome"],
                                                   random_state=42, shuffle=True, test_size=0.2)

In [24]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [25]:
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
60,2,84,0,0,0,0.0,0.304,21
618,9,112,82,24,0,28.2,1.282,50
346,1,139,46,19,83,28.7,0.654,22
294,0,161,50,0,0,21.9,0.254,65
231,6,134,80,37,370,46.2,0.238,46


In [29]:
scaler = MinMaxScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [33]:
clf = LinearSVC()
clf.fit(X_train_transformed, y_train)
y_pred = clf.predict(X_test_transformed)

In [38]:
pd.DataFrame(confusion_matrix(y_test, y_pred), columns=["Non-Diabetic", "Diabetic"], index = ["Non-Diabetic", "Diabetic"])

,Non-Diabetic,Diabetic
Non-Diabetic,81,18
Diabetic,18,37


In [42]:
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T

,precision,recall,f1-score,support
0,0.818182,0.818182,0.818182,99.000000
1,0.672727,0.672727,0.672727,55.000000
accuracy,0.766234,0.766234,0.766234,0.766234
macro avg,0.745455,0.745455,0.745455,154.000000
weighted avg,0.766234,0.766234,0.766234,154.000000


In [44]:
accuracy_score(y_test, y_pred)

0.7662337662337663

## Testing on few test dataset of users

In [45]:
input_data = (5,166,72,19,175,25.8,0.587,51)

np.array(input_data)

array([  5.   , 166.   ,  72.   ,  19.   , 175.   ,  25.8  ,   0.587,
        51.   ])

In [51]:
check_diabetic = {0: "Non Diabetic" , 1: "Diabetic"}
print("User is : {}".format(check_diabetic[clf.predict(scaler.transform([np.array(input_data)]))[0]]))

User is : Diabetic


C:\Users\ariji\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


## Downloading of the model

In [53]:
with open("diabetes_pred_model.pkl", "wb") as f:
    pickle.dump(clf, f)

## Load the file to check the prediction

In [54]:
with open("diabetes_pred_model.pkl", "rb") as f:
    clf_load = pickle.load(f)

In [55]:
clf_load.predict(scaler.transform([np.array(input_data)]))[0]

C:\Users\ariji\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1